# Math 124 - Programming for Mathematical Applications
UC Berkeley, Spring 2023

## Homework 8
Due Wednesday, March 22

In [30]:
using LinearAlgebra, PyPlot   # Packages needed

### Problem 1 - Cubic splines

Consider the interpolation of $n+1$ data points $(x_i,y_i)$, $i=0,\ldots,n$. A *cubic spline function* $S(x)$ is a piecewise cubic polynomial, that is, if $x_j\le x\le x_{j+1}$ then $S(x) = S_j(x)$ where

$$
\begin{align*}
S_j(x) = y_j + b_j(x-x_j)+c_j(x-x_j)^2+d_j(x-x_j)^3
\end{align*}
$$

The coefficients $b_j,c_j,d_j$, $j=0,\ldots,n-1$, are chosen such that the function is smooth and interpolates the given data.

#### Problem 1(a)

Write a function with the syntax `b,c,d = cubic_spline(x,y)`, which takes input data as vectors $\boldsymbol{x},\boldsymbol{y}$ and solves for the coefficient vectors $\boldsymbol{b},\boldsymbol{c},\boldsymbol{d}$ as described below. Create the matrix as a 
`Tridiagonal` matrix type in Julia, with the command `Tridiagonal(dl, d, du)` for lower-diagonal
`dl`, diagonal `d`, and upper-diagonal `du`.

Set $h_j=x_{j+1}-x_j$, $j=0,\ldots,n-1$, and solve the following linear system $A\boldsymbol{c}=\boldsymbol{f}$:

$$
\begin{align*}
A &=
\begin{pmatrix}
1 & 0 \\
h_0 & 2(h_0+h_1) & h_1 \\
    & \ddots & \ddots & \ddots \\
    &  & h_{n-2} & 2(h_{n-2}+h_{n-1}) & h_{n-1} \\
    &  &         & 0 & 1
\end{pmatrix} \\
\boldsymbol{f}&=(0,3(y_2-y_1)/h_1-3(y_1-y_0)/h_0,\ldots, \\
      &\ \ \ \ \ \ \ \ \ 3(y_n-y_{n-1})/h_{n-1}-3(y_{n-1}-y_{n-2})/h_{n-2},0)^T \\
\boldsymbol{c}&=(c_0,\ldots,c_n)^T
\end{align*}
$$

Here, $A \in \mathbb{R}^{(n+1) \times (n+1)}, f \in \mathbb{R}^{n+1}, c \in \mathbb{R}^{n+1}$,
which means `d` has size `n+1`, `dl` has size `n`, and `du` has size `n`.

Finally, compute the vectors $\boldsymbol{b},\boldsymbol{d}$ as

$$
\begin{align*}
b_j &= (y_{j+1}-y_j)/h_j-h_j(2c_j+c_{j+1})/3 \\
d_j &= (c_{j+1}-c_j)/(3h_j)
\end{align*}
$$

where $j=0,\ldots,n-1$.

In [141]:
function cubic_spline(x, y)
    n = length(x)
    h = diff(x)
    delta = zeros(n-1)
    for i in 1:n-1
        delta[i] = (y[i+1] - y[i])/h[i]
    end
    d = zeros(n)
    for i in 1:n-2
        d[i+1] = 2 * (h[i] + h[i+1])
    end
    lower = Float64.(h[1:end-1])
    diag = zeros(Float64, n)
    diag[1] = 1
    for i in 2:n-1
        diag[i] = 2 * (h[i-1] + h[i])
    end
    upper = Float64.(h[2:end])
    c = Tridiagonal(lower[1:end-1], diag[2:end-1], upper[1:end-1]) \ (3 .* delta)
    b = delta .* h .- h .* (2 .* c[1:end-1] .+ c[2:end]) ./ 3
    d[2:end-1] = (c[2:end] .- c[1:end-1]) ./ (3 .* h)
    return b, c, d
end

cubic_spline (generic function with 1 method)

#### Problem 1(b)

Write a function with the syntax `yy = spline_eval(x, y, b, c, d, xx)` which evaluates the spline defined by the data `x,y` and the computed coefficient vectors `b,c,d` at all the $x$-coordinates in `xx`.

In [32]:
function spline_eval(x, y, b, c, d, xx)
    n = length(x)
    yy = zeros(size(xx, 1))
    for i in 1:length(xx)
        j = 1
        while j < n && x[j+1] <= xx[i]
            j += 1
        end
        delt = xx[i] - x[j]
        yy[i] = y[j] + b[j]*delt + c[j]*delt^2 + d[j]*delt^3
    end
    return yy
end

spline_eval (generic function with 1 method)

#### Problem 1(c)

Test your function by computing the spline interpolant of the function

$$
f(x) = e^{-x/2}\sin 2x
$$

at the interpolation points $(x_i, f(x_i))$, $i=0,\ldots,10$.

Plot the original function and the spline interpolant on the interval $0\le x \le 10$.
Also plot markers at the interpolation points.

In [140]:
using Plots

x = range(0,stop=10, length=10)
f(x) = exp(-x/2) * sin(2x)
y = f.(x)

b, c, d = cubic_spline(x, y)
xx = range(0, stop=10, length=100)
yy = spline_eval(x, y, b, c, d, xx)

plot(xx, f.(xx), label="Original function")
scatter!(x, y, label="Interpolation points")
plot!(xx, yy, label="Spline interpolant")

LoadError: DimensionMismatch("tried to assign 9 elements to 8 destinations")

### Problem 2 - Parametric Spline Curves

To interpolate a set of data points using a *parametric spline curve*, we compute two piecewise cubic
polynomials $x(t)$ and $y(t)$, where $t$ is a parameter along the curve. For simplicity, we will
let $t_i = i$ for the $n+1$ data points $(x_i,y_i)$, $i=0,\ldots,n$, interpolated such that $x(t_i)=x_i$, $y(t_i)=y_i$.

#### Problem 2(a) - Plotting a parametric spline curve

Write a function with the syntax
```julia
function plot_parametric_spline(x,y; r=10)
```
which computes and plots a parametric spline for the points in the vectors `x,y`.

For the plotting, draw straight lines between the spline points $x(t),y(t)$ for $3r+1$ equally spaced values of $t$ between $0$ and $n$.

Draw the splines in blue, with a line-width of 0.5. Also set `axis` to `equal`.

Test the function using the code below.

In [5]:
xy = [0 0; 1 0; 1 1; 2 1; 2 0; 1 -1]
plot_parametric_spline(xy[:,1], xy[:,2])
plot(xy[:,1], xy[:,2], "ko");

LoadError: UndefVarError: plot_parametric_spline not defined

#### Problem 2(b) - Reading spline curves from a file

Download the file [`bmw.dat`](https://raw.githubusercontent.com/popersson/math124files/main/homework/bmw.dat), and upload it to the same directory as your Julia notebook.

The file contains the coordinates for a number of splines, with each spline in the following format:

$N_k$ <br>
$x_1$ $y_1$ <br>
$x_2$ $y_2$ <br>
$\ldots$ <br>
$x_{N_k}$ $y_{N_k}$


This is repeated until the file ends (which can be detected using the `eof` function). It is recommended to open the file and look at some of the lines, to make sure you know exactly how it is formatted.

Write a function

```julia
function read_splines(fname)
```

which opens a file `fname` containing splines as described above, and returns an array where the $k$-th element is an $N_k$-by-$2$
matrix with the $x,y$-points for each spline (note that $N_k$ is in general different for each spline).

*Hints*: This is probably easiest to do using strings and the `readline` function. To convert a string `str` to an integer, use `parse(Int64, str)`. To convert two numbers in the string to a vector of two floats, use `parse.(Float64, split(str))`.

Test your function by reading the file `bmw.dat`:

In [6]:
splines = read_splines("bmw.dat")

LoadError: UndefVarError: read_splines not defined

#### Problem 2(c) - Plotting an entire spline geometry

Write a function

```julia
function plot_splines(splines)
```

which plots all the parametric spline curves in `splines` (an array like the one returned by `read_splines`).

Plot the car twice using the commands below:

In [7]:
plot_splines(splines)

LoadError: UndefVarError: plot_splines not defined

In [8]:
plot_splines(splines);
axis([-121,-110.5,10,19]);

LoadError: UndefVarError: plot_splines not defined